In [14]:
from __future__ import annotations

from pathlib import Path
import re
import pandas as pd


# ----------------------------
# Helpers
# ----------------------------
_EN_TAIL_RE = re.compile(r"\s+[A-Z][a-zA-Z ,\-()']+$")

def _clean_tr_label(x: object) -> str:
    """
    Hücrede 'TR\nEN' gibi iki satır varsa ilk satırı alır,
    sonundaki İngilizce başlık kırpıntılarını da (varsayımsal) temizler.
    """
    if pd.isna(x):
        return ""
    s = str(x)
    s = s.split("\n")[0].strip()
    s = _EN_TAIL_RE.sub("", s).strip()
    return s


def read_gdp_raw_xls(path: str | Path) -> pd.DataFrame:
    """
    TÜİK il bazında GSYH (A10) zincirlenmiş hacim endeksi Excel’ini
    senin yaptığın header/ffill mantığıyla 'ham tablo' formatına getirir.
    """
    df = pd.read_excel(path)
    df = df.dropna(how="all", axis=1).dropna(how="all", axis=0)

    # Header satırları: sende 0-1-2 gibi görünüyor, 2. satırı ffill ediyordun.
    # Güvenli olsun diye: satır 2 yoksa patlamasın.
    if len(df) > 2:
        df.iloc[2] = df.iloc[2].ffill()

    # İlk 2 kolon (kod/il/yıl gibi) ffill
    if df.shape[1] >= 1:
        df.iloc[:, 0] = df.iloc[:, 0].ffill()
    if df.shape[1] >= 2:
        df.iloc[:, 1] = df.iloc[:, 1].ffill()

    # 2. satırdan itibaren devam
    df = df.iloc[2:].copy()

    # Kolon isimleri: ilk iki satır concat (sector--type gibi)
    # (df.iloc[0], df.iloc[1]) satırlarının varlığını kontrol edelim
    if len(df) < 2:
        raise ValueError("Beklenen header satırları bulunamadı (df çok kısa).")

    df.columns = df.iloc[0].astype("string") + "--" + df.iloc[1].astype("string")

    # Header satırlarını çıkar
    df = df.iloc[2:].copy()
    df = df.dropna(how="all")
    return df


def tidy_gdp_long(
    df: pd.DataFrame,
    id_cols: tuple[str, str, str] = ("code", "area", "year"),
) -> pd.DataFrame:
    """
    Ham tabloyu long formata çevirir:
    year/area/sector/type/value.
    """
    cols = list(df.columns)
    if len(cols) < 3:
        raise ValueError("Beklenen ilk 3 kolon (code/area/year) yok gibi görünüyor.")

    cols[:3] = list(id_cols)
    df = df.copy()
    df.columns = cols

    df = df.dropna()
    long = df.melt(id_vars=list(id_cols), var_name="variable", value_name="value")

    parts = long["variable"].astype("string").str.split("--", n=1, expand=True)
    long["sector"] = parts[0].map(_clean_tr_label)
    long["type"] = parts[1].map(_clean_tr_label)

    long = long.drop(columns=["variable", id_cols[0]], errors="ignore")  # code drop
    return long


def build_gdp_wide(
    long: pd.DataFrame,
    *,
    type_filter: str = "Hacim",
    sector_contains: str = "Gayrisafi",
    index_col: str = "year",
    columns_col: str = "area",
    values_col: str = "value",
    aggfunc: str = "sum",
) -> pd.DataFrame:
    """
    Filtre → pivot wide (year x area).
    """
    x = long.copy()

    # Filtreler
    x = x[(x["type"] == type_filter) & (x["sector"].astype("string").str.contains(sector_contains, na=False))]

    wide = (
        x.pivot_table(
            index=index_col,
            columns=columns_col,
            values=values_col,
            aggfunc=aggfunc,
        )
        .sort_index()
    )

    # year numeric ise numeric’e çevir (opsiyonel ama genelde faydalı)
    # çevrilemezse olduğu gibi bırakır
    try:
        wide.index = pd.to_numeric(wide.index)
        wide = wide.sort_index()
    except Exception:
        pass

    return wide


def compute_city_contributions(
    gdp_wide: pd.DataFrame,
    *,
    total_col: str = "Türkiye",
    drop_total_from_output: bool = True,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Türkiye büyümesi ve nominal değişim üzerinden şehir katkısı üretir.
    Dönüş:
      (gdp_with_total_metrics, city_contribution_percent)
    """
    df = gdp_wide.copy()

    if total_col not in df.columns:
        raise KeyError(f"'{total_col}' kolonu bulunamadı. Mevcut kolonlar: {list(df.columns)[:10]} ...")

    df["Türkiye_growth"] = df[total_col].pct_change()
    df["Türkiye_nominal_change"] = df[total_col].diff()

    city_cols = [c for c in df.columns if c not in ["Türkiye_growth", "Türkiye_nominal_change"]]

    city_nominal_change = df[city_cols].diff()

    # Bölme sırasında 0/NaN çıkabilir: inf’leri NaN’a çekelim
    contrib = (
        city_nominal_change
        .div(df["Türkiye_nominal_change"], axis=0)
        .mul(df["Türkiye_growth"], axis=0)
        .replace([float("inf"), float("-inf")], pd.NA)
    )

    if drop_total_from_output and total_col in contrib.columns:
        contrib = contrib.drop(columns=[total_col])

    contrib_percent = contrib.mul(100)
    return df, contrib_percent


def run_pipeline(
    input_xls: str | Path,
    output_xlsx: str | Path,
    *,
    type_filter: str = "Hacim",
    sector_contains: str = "Gayrisafi",
    total_col: str = "Türkiye",
) -> pd.DataFrame:
    """
    Baştan sona çalıştırır ve output’a yazar.
    Dönüş: şehir katkı % DataFrame’i (export edilen).
    """
    raw = read_gdp_raw_xls(input_xls)
    long = tidy_gdp_long(raw)
    wide = build_gdp_wide(long, type_filter=type_filter, sector_contains=sector_contains)
    _, contrib_percent = compute_city_contributions(wide, total_col=total_col)

    output_xlsx = Path(output_xlsx)
    output_xlsx.parent.mkdir(parents=True, exist_ok=True)
    contrib_percent.to_excel(output_xlsx)

    return contrib_percent


# ----------------------------
# Usage
# ----------------------------
contrib_percent = run_pipeline(
    input_xls="data/il bazinda gayrisafi yurt ici hasila iktisadi faaliyet kollarina (a10) gore zincirlenmis hacim endek.xls",
    output_xlsx="results/gdp.xlsx",
    type_filter="Hacim",
    sector_contains="Gayrisafi",
    total_col="Türkiye",
)

contrib_percent


/var/folders/lx/dlxz3kpj1pq9fx9tbtzznnr00000gn/T/ipykernel_8944/2866015970.py:143: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Türkiye_growth"] = df[total_col].pct_change()
/var/folders/lx/dlxz3kpj1pq9fx9tbtzznnr00000gn/T/ipykernel_8944/2866015970.py:155: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace([float("inf"), float("-inf")], pd.NA)


area,Adana,Adıyaman,Afyonkarahisar,Aksaray,Amasya,Ankara,Antalya,Ardahan,Artvin,Aydın,...,Yalova,Yozgat,Zonguldak,Çanakkale,Çankırı,Çorum,İstanbul,İzmir,Şanlıurfa,Şırnak
year,,,,,,,,,,,,,,,,,,,,,
2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001,-0.085486,-0.004257,-0.037277,-0.051275,-0.023015,-0.400700,-0.123211,-0.003868,-0.009108,-0.104649,...,-0.018290,-0.040079,-0.033526,-0.057715,-0.012878,-0.046400,-1.392991,-0.375681,0.022913,0.003277
2002,0.153879,0.009796,0.043888,0.048168,0.032819,0.513613,0.192659,0.006921,0.011009,0.154534,...,0.016605,0.053935,0.035956,0.068148,0.010765,0.046889,1.732385,0.440133,0.036810,0.017568
2003,0.091751,0.015811,0.049502,0.013447,0.007058,0.536601,0.161114,0.007363,0.013919,-0.070769,...,0.023598,0.012323,0.035633,0.004130,0.010600,0.010846,2.040928,0.321160,0.049700,0.012086
2004,0.175121,0.013919,0.053368,0.026026,0.022011,0.939884,0.330197,-0.003222,0.020659,0.204286,...,0.030858,0.026833,0.061870,0.081246,0.013682,0.062881,3.456687,0.763184,0.049367,0.015959
2005,0.179491,0.030858,0.031279,0.024637,0.028910,0.838592,0.252288,0.005843,0.017325,0.039000,...,0.020685,0.020188,0.051831,0.055931,0.010238,0.023404,2.962891,0.507193,0.050986,0.015943
2006,0.124018,0.022570,0.048019,0.020834,0.005763,0.776392,0.248841,0.010207,0.019008,0.113556,...,0.035640,0.003519,0.041298,0.024546,-0.002549,0.002690,2.181139,0.471509,0.052267,0.014164
2007,0.110516,0.012544,0.017815,0.013745,-0.002435,0.641094,0.169628,0.000797,0.012881,-0.040416,...,0.017650,0.010915,0.015598,0.026613,0.004594,0.011195,1.994824,0.254705,0.011215,0.018885
2008,-0.004941,0.004195,0.006940,0.003599,0.022427,0.142920,-0.043089,-0.001525,0.003621,0.012485,...,0.011250,-0.007888,-0.024673,-0.005360,0.006699,0.028311,0.250761,-0.003844,0.001555,0.004932


,City,Contribution_percent
0,İstanbul,0.006233
1,Ankara,0.004433
2,Hatay,0.001649
3,Gaziantep,0.001432
4,Mersin,0.001413
...,...,...
76,Bilecik,-0.000063
77,Mardin,-0.000081
78,Adana,-0.00011
79,Kırşehir,-0.000125


In [15]:
from __future__ import annotations

from pathlib import Path
import re
import pandas as pd


# =============================================================================
# CONFIG
# =============================================================================
INPUT_XLS = Path("data/il bazinda gayrisafi yurt ici hasila iktisadi faaliyet kollarina (a10) gore zincirlenmis hacim endek.xls")
OUTPUT_XLSX = Path("results/gdp_city_contributions.xlsx")

TYPE_FILTER = "Hacim"
SECTOR_CONTAINS = "Gayrisafi"   # senin filtreden
TOTAL_COL = "Türkiye"

# 2023 Kahramanmaraş merkezli depremden etkilenen iller (yaygın liste)
EARTHQUAKE_CITIES = [
    "Kahramanmaraş",
    "Hatay",
    "Gaziantep",
    "Adıyaman",
    "Malatya",
    "Adana",
    "Osmaniye",
    "Kilis",
    "Diyarbakır",
    "Şanlıurfa",
    "Elazığ",
]


# =============================================================================
# Helpers
# =============================================================================
_EN_TAIL_RE = re.compile(r"\s+[A-Z][a-zA-Z ,\-()']+$")


def _clean_tr_label(x: object) -> str:
    """TR\\nEN gibi hücrelerden TR kısmını alır + sonundaki İngilizce kırpıntıları temizler."""
    if pd.isna(x):
        return ""
    s = str(x)
    s = s.split("\n")[0].strip()
    s = _EN_TAIL_RE.sub("", s).strip()
    return s


def read_gdp_raw_xls(path: str | Path) -> pd.DataFrame:
    """Exceli senin mantığınla header/ffill ederek ham tabloya çevirir."""
    df = pd.read_excel(path)
    df = df.dropna(how="all", axis=1).dropna(how="all", axis=0)

    # satır 2 ffill
    if len(df) > 2:
        df.iloc[2] = df.iloc[2].ffill()

    # ilk iki kolon ffill
    if df.shape[1] >= 1:
        df.iloc[:, 0] = df.iloc[:, 0].ffill()
    if df.shape[1] >= 2:
        df.iloc[:, 1] = df.iloc[:, 1].ffill()

    df = df.iloc[2:].copy()

    if len(df) < 2:
        raise ValueError("Beklenen header satırları bulunamadı (df çok kısa).")

    df.columns = df.iloc[0].astype("string") + "--" + df.iloc[1].astype("string")

    # header satırlarını çıkar
    df = df.iloc[2:].copy()
    df = df.dropna(how="all")
    return df


def tidy_gdp_long(df: pd.DataFrame) -> pd.DataFrame:
    """Ham tabloyu long formata (year/area/sector/type/value) çevirir."""
    cols = list(df.columns)
    if len(cols) < 3:
        raise ValueError("İlk 3 kolon bekleniyordu (code/area/year).")

    cols[:3] = ["code", "area", "year"]
    x = df.copy()
    x.columns = cols

    x = x.dropna()
    long = x.melt(id_vars=["code", "area", "year"], var_name="variable", value_name="value")

    parts = long["variable"].astype("string").str.split("--", n=1, expand=True)
    long["sector"] = parts[0].map(_clean_tr_label)
    long["type"] = parts[1].map(_clean_tr_label)

    long = long.drop(columns=["variable", "code"], errors="ignore")
    return long


def build_gdp_wide(long: pd.DataFrame, *, type_filter: str, sector_contains: str) -> pd.DataFrame:
    """Filtre → pivot wide (year x area)."""
    x = long.copy()

    x = x[
        (x["type"] == type_filter)
        & (x["sector"].astype("string").str.contains(sector_contains, na=False))
    ].copy()

    wide = (
        x.pivot_table(
            index="year",
            columns="area",
            values="value",
            aggfunc="sum",
        )
        .sort_index()
    )

    # year index numeric olabiliyorsa dönüştür (sorting iyi olsun)
    try:
        wide.index = pd.to_numeric(wide.index)
        wide = wide.sort_index()
    except Exception:
        pass

    return wide


def compute_city_contributions(gdp_wide: pd.DataFrame, *, total_col: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Türkiye büyümesi ve nominal değişim ile şehir katkılarını hesaplar.
    Dönüş:
      - gdp_wide_with_metrics (Türkiye_growth, Türkiye_nominal_change ekli)
      - city_contribution_percent (şehir kolonları, % katkı)
    """
    df = gdp_wide.copy()

    if total_col not in df.columns:
        raise KeyError(f"'{total_col}' kolonu yok. Kolon örneği: {list(df.columns)[:10]}")

    df["Türkiye_growth"] = df[total_col].pct_change()
    df["Türkiye_nominal_change"] = df[total_col].diff()

    city_cols = [c for c in df.columns if c not in ["Türkiye_growth", "Türkiye_nominal_change"]]

    city_nominal_change = df[city_cols].diff()

    contrib = (
        city_nominal_change
        .div(df["Türkiye_nominal_change"], axis=0)
        .mul(df["Türkiye_growth"], axis=0)
        .replace([float("inf"), float("-inf")], pd.NA)
    )

    # Türkiye'yi şehir katkılarından çıkar (istersen dahil edebilirsin)
    contrib = contrib.drop(columns=[total_col], errors="ignore")

    contrib_percent = contrib.mul(100)
    return df, contrib_percent


def last_year_city_table(city_contribution_percent: pd.DataFrame) -> tuple[int | str, pd.DataFrame]:
    """Son yılın tüm şehir katkılarını 'City/Contribution_percent' tablosu olarak döndürür."""
    last_year = city_contribution_percent.index.max()
    s = city_contribution_percent.loc[last_year].dropna().sort_values(ascending=False)
    out = s.reset_index()
    out.columns = ["City", "Contribution_percent"]
    return last_year, out


def earthquake_summary(
    gdp_with_metrics: pd.DataFrame,
    city_contribution_percent: pd.DataFrame,
    *,
    earthquake_cities: list[str],
    total_col: str,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Son yıl için deprem illeri:
      - iller bazında katkı tablosu
      - toplam katkı (yüzde puan) + Türkiye büyümesi + pay
    """
    last_year = city_contribution_percent.index.max()

    last_contrib = city_contribution_percent.loc[last_year]
    available = [c for c in earthquake_cities if c in last_contrib.index]
    missing = [c for c in earthquake_cities if c not in last_contrib.index]

    eq_table = (
        last_contrib.loc[available]
        .dropna()
        .sort_values(ascending=False)
        .reset_index()
    )
    eq_table.columns = ["City", "Contribution_percent"]

    eq_total = eq_table["Contribution_percent"].sum()

    turkey_growth_percent = float(gdp_with_metrics.loc[last_year, "Türkiye_growth"] * 100)
    eq_share = (eq_total / turkey_growth_percent * 100) if turkey_growth_percent not in (0.0, -0.0) else float("nan")

    summary = pd.DataFrame(
        {
            "metric": [
                "year",
                "earthquake_total_contribution_percent_point",
                "turkey_growth_percent",
                "earthquake_share_of_growth_percent",
                "earthquake_cities_used_count",
                "earthquake_cities_missing_count",
                "earthquake_cities_missing_list",
            ],
            "value": [
                last_year,
                eq_total,
                turkey_growth_percent,
                eq_share,
                len(available),
                len(missing),
                ", ".join(missing),
            ],
        }
    )

    return eq_table, summary


# =============================================================================
# Run
# =============================================================================
def main() -> None:
    OUTPUT_XLSX.parent.mkdir(parents=True, exist_ok=True)

    raw = read_gdp_raw_xls(INPUT_XLS)
    long = tidy_gdp_long(raw)
    gdp_wide = build_gdp_wide(long, type_filter=TYPE_FILTER, sector_contains=SECTOR_CONTAINS)

    gdp_with_metrics, city_contrib_percent = compute_city_contributions(gdp_wide, total_col=TOTAL_COL)

    # Son yıl tüm şehir tablosu
    last_year, city_table = last_year_city_table(city_contrib_percent)

    # Deprem özeti
    eq_table, eq_summary = earthquake_summary(
        gdp_with_metrics=gdp_with_metrics,
        city_contribution_percent=city_contrib_percent,
        earthquake_cities=EARTHQUAKE_CITIES,
        total_col=TOTAL_COL,
    )

    # Export (multi-sheet)
    with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as writer:
        gdp_wide.to_excel(writer, sheet_name="gdp_wide")
        gdp_with_metrics.to_excel(writer, sheet_name="gdp_with_metrics")
        city_contrib_percent.to_excel(writer, sheet_name="city_contrib_percent_all_years")

        city_table.to_excel(writer, sheet_name=f"cities_{last_year}", index=False)
        eq_table.to_excel(writer, sheet_name=f"eq_cities_{last_year}", index=False)
        eq_summary.to_excel(writer, sheet_name=f"eq_summary_{last_year}", index=False)

    print("Saved:", OUTPUT_XLSX)
    print("Last year:", last_year)
    print("Earthquake total contribution (pp):", float(eq_summary.loc[eq_summary.metric=="earthquake_total_contribution_percent_point","value"].iloc[0]))


if __name__ == "__main__":
    main()


Saved: results/gdp_city_contributions.xlsx
Last year: 2024
Earthquake total contribution (pp): 0.8942182503772138


/var/folders/lx/dlxz3kpj1pq9fx9tbtzznnr00000gn/T/ipykernel_8944/3414839181.py:140: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Türkiye_growth"] = df[total_col].pct_change()
/var/folders/lx/dlxz3kpj1pq9fx9tbtzznnr00000gn/T/ipykernel_8944/3414839181.py:151: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace([float("inf"), float("-inf")], pd.NA)
